### Название основных параметров 
1. `ROP` – rate of penetration или скорость бурения (м/ч). Тот самый параметр, который определят в конечном итоге продолжительность бурения.
2. `WOB` – weight on bit или нагрузка на долото (тонн). При сверлении стен – это усилие которые вы прикладываете к дрели.
3. `RPM` – revolution per minute или скорость вращения долота. На ручной дрели вы так же этот параметр можете отрегулировать (больше/меньше).
4. `SPP` – stand pipe pressure или давление под которым специальный раствор подается в бурильную трубу на поверхности для того чтобы выносить/вымывать из скважины породу (шлам) разрушенную долотом (пыль, которая получается при сверлении стены).
5. `Flow_in` – это расход того самого специального раствора для вымыва шлама из скважины, измеряется в л/мин.
6. `Torque` – крутящий момент. Самый сложный для понимания параметр. Это расчетная величина (М), которая характеризует действие приложенного усилия для создания вращения долота.
7. `BDT` – bit drill time или время эксплуатации долота.
8. `MD` – measured depth или измеренная глубина. Это расстояние от площадки на поверхности (где стоит буровик и что-то делает) и долотом. Используется для измерения расстояния от поверхности до границ пластов и других объектов под землей, измеряется в метрах. Так как скважины не всегда идеально прямые, то эта величина всегда немного больше вертикального расстояния.
9. `MD_form` – расстояние от кровли (верхней границы) пласта. Гораздо более полезная для анализа величина. В силу неровности земной поверхности расстояние от поверхности до кровли пласта может быть разное. (MD_1 < MD_2). При этом, в силу принципов формирования отложений горной породы, мы понимаем, что с большой долей вероятности те или иные породы будут расположены примерно на одинаковом расстоянии от кровли пласта (MD_form_1 == MD_form_2) во всех рассматриваемых скважинах. Так это работает. Поэтому для анализа нам желательно анализировать параметры бурения в одинаковых по свойству породах и расстояние от кровли пласта нам поможет в данном случае.

In [1]:
import pandas as pd 
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from catboost import Pool, CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from datetime import datetime

In [3]:
df = pd.read_csv("Clear_data.csv", index_col = 0)

In [17]:
## Выбор модели
# для пласта i = 0
all_models = [LinearRegression(), Lasso(), Ridge(), BayesianRidge(), 
              DecisionTreeRegressor(),
              GradientBoostingRegressor(), RandomForestRegressor(),
              CatBoostRegressor(iterations=260, 
                          depth=2, 
                          learning_rate=1, 
                          loss_function='RMSE',
                               verbose=False)
             ]

names = ['Линейная регрессия', 'Лассо регрессия', 'Ридж регрессия', 'Байесовская регрессия',
         'Решающее дерево',
         'Градиентый бустинг', 'Случайный лес', 
         'Катбуст'
        ]
r2_train = []
r2_test = []
plast = []
time = []
metrics_train = []
metrics_test = []
models = []
preprocessing = []
optimization = []
for j, specific_model in enumerate(all_models): 
    for i in df['plast'].unique():
        df_i = df[df['plast'] == i].reset_index(drop=True).drop(['Типоразмер штанги', 'plast'], axis = 1)
        y = df_i['Мгновенная механическая скорость бурения']
        x = df_i.drop('Мгновенная механическая скорость бурения', axis = 1)
        X_train, X_test, y_train, y_test = train_test_split(x, y, train_size= 0.9, random_state=123, shuffle=True)
        std = StandardScaler()
        X_train = std.fit_transform(X_train)
        X_test = stf.transform(X_test)
        model = specific_model
        start_time = datetime.now()
        model.fit(X_train, y_train)
        end_time = datetime.now()
        time.append((end_time - start_time).total_seconds())
        y_pred_test = model.predict(X_test)
        y_pred_train = model.predict(X_train)
        #print(f'RMSE for train: {mean_squared_error(y_train, y_pred_train) ** 0.5}')
        #print(f'RMSE for test: {mean_squared_error(y_test, y_pred_test) ** 0.5}')
        r2_train.append(r2_score(y_train, y_pred_train))
        r2_test.append(r2_score(y_test, y_pred_test))
        models.append(names[j])
        plast.append(i)
        metrics_train.append(mean_squared_error(y_train, y_pred_train) ** 0.5)
        metrics_test.append(mean_squared_error(y_test, y_pred_test) ** 0.5)
        preprocessing.append('0')
        optimization.append('0')

In [18]:
metrics_all = pd.DataFrame({'Пласт': plast, 'Модель': models, 
                            'Ошибка на трейне': metrics_train, 'Ошибка на тесте': metrics_test,
                            'Время': [i for i in time],
                           'Обработка': [(i) for i in preprocessing], 
                            'Оптимизация': [(i) for i in optimization],
                            'R2_трейн': r2_train,
                            'R2_тест': r2_test
                           })

In [20]:
pd.options.display.float_format = '{:.3f}'.format
metrics_all.groupby(['Пласт', 'Модель']).mean()

                             Ошибка на трейне  Ошибка на тесте  Время  \
Пласт Модель                                                            
0     Байесовская регрессия             7.941            9.417  0.007   
      Градиентый бустинг                2.465            2.823  0.595   
      Катбуст                           1.670            2.178  0.477   
      Лассо регрессия                   7.949            9.415  0.006   
      Линейная регрессия                7.941            9.419  0.036   
      Решающее дерево                   0.000            1.231  0.031   
      Ридж регрессия                    7.941            9.419  0.007   
      Случайный лес                     0.932            1.574  1.731   
1     Байесовская регрессия             2.449            2.446  0.003   
      Градиентый бустинг                0.866            1.321  0.153   
      Катбуст                           0.349            1.067  0.142   
      Лассо регрессия                   2.481      

In [27]:
import dataframe_image as di 
di.export(metrics_all.groupby(['Пласт', 'Модель']).mean(), 'gg.png')

In [32]:
## Выбор модели
# для пласта i = 0
from sklearn.linear_model import LinearRegression, Lasso, Ridge, BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from catboost import Pool, CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from datetime import datetime
all_models = [LinearRegression(), Lasso(), Ridge(), BayesianRidge(), 
              DecisionTreeRegressor(),
              GradientBoostingRegressor(), RandomForestRegressor(),
              CatBoostRegressor(iterations=260, 
                          depth=2, 
                          learning_rate=1, 
                          loss_function='RMSE',
                               verbose=False)
             ]
names = ['Линейная регрессия', 'Лассо регрессия', 'Ридж регрессия', 'Байесовская регрессия',
         'Решающее дерево',
         'Градиентый бустинг', 'Случайный лес', 
         'Катбуст'
        ]
r2_train = []
r2_test = []
plast = []
time = []
metrics_train = []
metrics_test = []
models = []
preprocessing = []
optimization = []
for j, specific_model in enumerate(all_models): 
    for i in df['plast'].unique():
        df_i = df[df['plast'] == i].reset_index(drop=True).drop(['Типоразмер штанги', 'plast'], axis = 1)
        y = df_i['Мгновенная механическая скорость бурения']
        x = df_i.drop(['Мгновенная механическая скорость бурения', 'Давление промывочной жидкости', 
                       'Крутящий момент на шпинделе', 'Глубина скважины (м)'], axis = 1)
        X_train, X_test, y_train, y_test = train_test_split(x, y, train_size= 0.9, random_state=123, shuffle=True)
        model = specific_model
        start_time = datetime.now()
        model.fit(X_train, y_train)
        end_time = datetime.now()
        time.append((end_time - start_time).total_seconds())
        y_pred_test = model.predict(X_test)
        y_pred_train = model.predict(X_train)
        #print(f'RMSE for train: {mean_squared_error(y_train, y_pred_train) ** 0.5}')
        #print(f'RMSE for test: {mean_squared_error(y_test, y_pred_test) ** 0.5}')
        r2_train.append(r2_score(y_train, y_pred_train))
        r2_test.append(r2_score(y_test, y_pred_test))
        models.append(names[j])
        plast.append(i)
        metrics_train.append(mean_squared_error(y_train, y_pred_train) ** 0.5)
        metrics_test.append(mean_squared_error(y_test, y_pred_test) ** 0.5)
        preprocessing.append('0')
        optimization.append('0')

In [120]:
metrics_all = pd.DataFrame({'Пласт': plast, 'Модель': models, 
                            'Ошибка на трейне': metrics_train, 'Ошибка на тесте': metrics_test,
                            'Время': [i for i in time],
                           'Обработка': [(i) for i in preprocessing], 
                            'Оптимизация': [(i) for i in optimization],
                            'R2_трейн': r2_train,
                            'R2_тест': r2_test
                           })
z = metrics_all.groupby(['Пласт', 'Модель']).mean()

In [52]:
df_i = df[df['plast'] == i].reset_index(drop=True).drop(['Типоразмер штанги', 'plast'], axis = 1)
y = df_i['Мгновенная механическая скорость бурения']
x = df_i.drop(['Мгновенная механическая скорость бурения', 'Давление промывочной жидкости', 
               'Крутящий момент на шпинделе', 'Глубина скважины (м)', 'flight_cat'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size= 0.9, random_state=123, shuffle=True)
model = all_models[-1]
model.fit(X_train, y_train)